# Part 7.2: AI Agents and Tool Use

An LLM that can only generate text is like a brilliant advisor locked in a room — full of knowledge but unable to *do* anything. **AI agents** break out of this limitation by giving LLMs the ability to take actions: search the web, execute code, call APIs, and interact with the world through **tools**.

Agent architectures are behind the most capable AI systems today — from coding assistants that edit files and run tests, to research agents that browse the web and synthesize findings. Understanding agents means understanding how AI goes from "answering questions" to "completing tasks."

## Learning Objectives

- [ ] Understand the agent paradigm: observe, reason, act, repeat
- [ ] Implement tool definitions and a tool execution framework
- [ ] Build a ReAct (Reasoning + Acting) agent from scratch
- [ ] Understand chain-of-thought reasoning and why it improves agent performance
- [ ] Implement multi-step planning and execution
- [ ] Build a simple code-execution agent
- [ ] Understand agent failure modes and safety considerations
- [ ] Compare single-turn vs. multi-turn agent architectures

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import json
import re
import math
from collections import defaultdict
from datetime import datetime

np.random.seed(42)

print("Part 7.2: AI Agents and Tool Use")
print("=" * 50)

---

## 1. What is an AI Agent?

An **agent** is a system that uses an LLM as its reasoning engine to:
1. **Observe** the current state (user request, tool outputs, environment)
2. **Reason** about what to do next (chain-of-thought)
3. **Act** by calling tools or generating responses
4. **Repeat** until the task is complete

### Agent vs. Chatbot

| Feature | Chatbot | Agent |
|---------|---------|-------|
| **Interaction** | Single response per turn | Multi-step execution |
| **Tools** | None (text only) | Can call APIs, run code, search |
| **Planning** | React to each message | Plan ahead, decompose tasks |
| **State** | Conversation history | + tool outputs, environment state |
| **Autonomy** | User drives every step | Agent decides what to do next |

### Visualization: The Agent Loop

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.set_xlim(0, 12)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('The AI Agent Loop', fontsize=16, fontweight='bold')

# Central LLM
circle = plt.Circle((6, 5), 1.5, color='#3498db', alpha=0.9, zorder=5)
ax.add_patch(circle)
ax.text(6, 5.2, 'LLM', ha='center', va='center', fontsize=16,
        fontweight='bold', color='white')
ax.text(6, 4.6, '(Reasoning\nEngine)', ha='center', va='center', fontsize=9, color='white')

# Surrounding components
components = [
    (2, 8, '1. Observe', '#2ecc71', 'Read user request\n+ tool outputs'),
    (10, 8, '2. Reason', '#9b59b6', 'Chain-of-thought\nplanning'),
    (10, 2, '3. Act', '#e74c3c', 'Call tools or\ngenerate response'),
    (2, 2, '4. Update', '#f39c12', 'Add result to\ncontext'),
]

for x, y, label, color, desc in components:
    box = mpatches.FancyBboxPatch((x - 1.2, y - 0.5), 2.4, 1, boxstyle="round,pad=0.2",
                                   facecolor=color, edgecolor='black', linewidth=2, alpha=0.9)
    ax.add_patch(box)
    ax.text(x, y, label, ha='center', va='center', fontsize=10,
            fontweight='bold', color='white')
    ax.text(x, y - 0.9, desc, ha='center', va='center', fontsize=8, color='gray')

# Arrows forming a cycle
arrow_pairs = [(3.2, 8, 8.8, 8), (10, 7.5, 10, 2.5),
               (8.8, 2, 3.2, 2), (2, 2.5, 2, 7.5)]
for x1, y1, x2, y2 in arrow_pairs:
    ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
               arrowprops=dict(arrowstyle='->', lw=2.5, color='gray'))

# Tools
tools = ['Search', 'Calculator', 'Code Exec', 'API Call']
for i, tool in enumerate(tools):
    x = 5 + i * 1.8
    ax.text(x, 0.5, f'🔧 {tool}', ha='center', fontsize=9,
            bbox=dict(boxstyle='round', facecolor='#ecf0f1', edgecolor='gray'))

ax.text(7.8, 1.2, 'Available Tools', ha='center', fontsize=10,
        fontweight='bold', color='gray')

plt.tight_layout()
plt.show()

---

## 2. Tool Definitions

Tools are the agent's interface to the world. Each tool has:
- A **name** and **description** (so the LLM knows when to use it)
- **Parameters** with types and descriptions
- An **execute** function that runs the tool

This is the same format used by OpenAI function calling, Anthropic tool use, and most agent frameworks.

In [ ]:
class Tool:
    """Base class for agent tools."""
    
    def __init__(self, name, description, parameters):
        self.name = name
        self.description = description
        self.parameters = parameters  # Dict of {param_name: {type, description}}
    
    def execute(self, **kwargs):
        raise NotImplementedError
    
    def schema(self):
        """Return tool schema (like OpenAI function calling format)."""
        return {
            'name': self.name,
            'description': self.description,
            'parameters': self.parameters
        }


class CalculatorTool(Tool):
    """Evaluate mathematical expressions safely."""
    
    def __init__(self):
        super().__init__(
            name='calculator',
            description='Evaluate a mathematical expression. Supports +, -, *, /, **, sqrt, sin, cos, log.',
            parameters={'expression': {'type': 'string', 'description': 'Math expression to evaluate'}}
        )
    
    def execute(self, expression):
        """Safely evaluate a math expression."""
        # Whitelist of safe operations
        allowed = {'sqrt': math.sqrt, 'sin': math.sin, 'cos': math.cos,
                   'log': math.log, 'pi': math.pi, 'e': math.e,
                   'abs': abs, 'round': round, 'pow': pow}
        try:
            result = eval(expression, {"__builtins__": {}}, allowed)
            return {'status': 'success', 'result': result}
        except Exception as e:
            return {'status': 'error', 'error': str(e)}


class SearchTool(Tool):
    """Simulated web search tool."""
    
    def __init__(self):
        super().__init__(
            name='search',
            description='Search for information on a topic. Returns relevant snippets.',
            parameters={'query': {'type': 'string', 'description': 'Search query'}}
        )
        # Simulated search index
        self.knowledge = {
            'transformer': 'The Transformer was introduced in 2017 by Vaswani et al. in "Attention Is All You Need". It uses self-attention instead of recurrence.',
            'attention': 'Self-attention computes Query, Key, Value matrices. Attention(Q,K,V) = softmax(QK^T/sqrt(d_k))V.',
            'rlhf': 'RLHF uses three stages: SFT on demonstrations, reward model training on preferences, and PPO optimization.',
            'backpropagation': 'Backpropagation computes gradients using the chain rule, propagating error backwards through the network.',
            'gpt': 'GPT models are decoder-only transformers trained with autoregressive language modeling. GPT-4 was released in March 2023.',
            'embedding': 'Embeddings map discrete tokens to dense vectors. Modern sentence embeddings capture semantic meaning.',
            'python': 'Python is a high-level programming language known for its readability. It is widely used in AI/ML.',
            'pytorch': 'PyTorch is an open-source deep learning framework developed by Meta. It uses dynamic computation graphs.',
        }
    
    def execute(self, query):
        """Simulate searching for information."""
        query_lower = query.lower()
        results = []
        for key, value in self.knowledge.items():
            if key in query_lower or any(word in query_lower for word in key.split()):
                results.append({'title': key.title(), 'snippet': value})
        
        if not results:
            return {'status': 'success', 'results': [], 'message': 'No results found.'}
        return {'status': 'success', 'results': results[:3]}


class LookupTool(Tool):
    """Look up a specific fact from a knowledge base."""
    
    def __init__(self):
        super().__init__(
            name='lookup',
            description='Look up a specific fact or definition.',
            parameters={'term': {'type': 'string', 'description': 'Term to look up'}}
        )
        self.facts = {
            'relu': 'ReLU(x) = max(0, x). Most common activation function in deep learning.',
            'softmax': 'softmax(x_i) = exp(x_i) / sum(exp(x_j)). Converts logits to probabilities.',
            'adam': 'Adam optimizer combines momentum and RMSprop. Default lr=0.001, betas=(0.9, 0.999).',
            'cross entropy': 'Cross-entropy loss: L = -sum(y_i * log(p_i)). Standard for classification.',
            'batch normalization': 'Normalizes layer inputs to zero mean, unit variance. Speeds up training.',
            'dropout': 'Randomly zeros elements with probability p during training. Regularization technique.',
        }
    
    def execute(self, term):
        term_lower = term.lower().strip()
        if term_lower in self.facts:
            return {'status': 'success', 'definition': self.facts[term_lower]}
        # Fuzzy match
        for key, value in self.facts.items():
            if term_lower in key or key in term_lower:
                return {'status': 'success', 'definition': value}
        return {'status': 'not_found', 'message': f'No definition found for "{term}"'}


# Create our tool registry
tools = {
    'calculator': CalculatorTool(),
    'search': SearchTool(),
    'lookup': LookupTool(),
}

print("Available tools:")
for name, tool in tools.items():
    print(f"  {name}: {tool.description}")

# Test tools
print("\nTool tests:")
print(f"  calculator('2**10'): {tools['calculator'].execute(expression='2**10')}")
print(f"  search('transformer'): {tools['search'].execute(query='transformer architecture')}")
print(f"  lookup('relu'): {tools['lookup'].execute(term='relu')}")

---

## 3. The ReAct Pattern

**ReAct** (Reasoning + Acting) interleaves chain-of-thought reasoning with tool use:

```
Thought: I need to find out when the Transformer was introduced.
Action: search("transformer architecture origin")
Observation: The Transformer was introduced in 2017...
Thought: Now I know it was 2017. Let me calculate how many years ago that was.
Action: calculator("2026 - 2017")
Observation: 9
Thought: I have all the information I need.
Answer: The Transformer was introduced 9 years ago, in 2017.
```

The key insight: by verbalizing its reasoning, the agent makes better decisions about *which* tool to use and *when*.

In [ ]:
class ReActAgent:
    """ReAct agent: Reasoning + Acting with tool use.
    
    Uses a simulated LLM (rule-based) to demonstrate the pattern.
    In production, this would be an actual LLM like Claude or GPT-4.
    """
    
    def __init__(self, tools, max_steps=5):
        self.tools = tools
        self.max_steps = max_steps
        self.trace = []  # Full execution trace
    
    def _simulate_reasoning(self, query, observations):
        """Simulate LLM reasoning to decide next action.
        
        In production, this is where you'd call the LLM API.
        Here we use heuristics to demonstrate the pattern.
        """
        query_lower = query.lower()
        
        # If we already have observations, try to answer
        if len(observations) >= 2:
            return {'type': 'answer', 'content': self._synthesize(query, observations)}
        
        # Decide which tool to use based on query
        if any(word in query_lower for word in ['calculate', 'compute', 'how many', 'what is', 'evaluate'])\
           and any(c in query for c in '0123456789+-*/^'):
            # Extract math expression
            expr = re.findall(r'[\d\+\-\*/\(\)\. \*\*]+', query)
            if expr:
                expression = expr[0].strip()
                return {
                    'type': 'action',
                    'thought': f'I need to calculate {expression}.',
                    'tool': 'calculator',
                    'args': {'expression': expression}
                }
        
        if any(word in query_lower for word in ['what is', 'define', 'explain']):
            # Try lookup first
            terms = re.findall(r'what is (\w+(?:\s+\w+)?)', query_lower)
            if terms and not observations:
                return {
                    'type': 'action',
                    'thought': f'Let me look up the definition of "{terms[0]}".',
                    'tool': 'lookup',
                    'args': {'term': terms[0]}
                }
        
        # Default: search
        if not observations:
            return {
                'type': 'action',
                'thought': f'I need to search for information about this topic.',
                'tool': 'search',
                'args': {'query': query}
            }
        
        return {'type': 'answer', 'content': self._synthesize(query, observations)}
    
    def _synthesize(self, query, observations):
        """Synthesize a final answer from observations."""
        parts = []
        for obs in observations:
            if isinstance(obs.get('result'), dict):
                if 'results' in obs['result']:
                    for r in obs['result']['results']:
                        parts.append(r.get('snippet', ''))
                elif 'definition' in obs['result']:
                    parts.append(obs['result']['definition'])
                elif 'result' in obs['result']:
                    parts.append(f"Calculation result: {obs['result']['result']}")
        
        return ' '.join(parts) if parts else 'I could not find enough information to answer.'
    
    def run(self, query):
        """Execute the ReAct loop."""
        self.trace = []
        observations = []
        
        self.trace.append({'step': 'query', 'content': query})
        
        for step in range(self.max_steps):
            # Reason about what to do
            decision = self._simulate_reasoning(query, observations)
            
            if decision['type'] == 'answer':
                self.trace.append({
                    'step': 'answer',
                    'thought': 'I have enough information to answer.',
                    'content': decision['content']
                })
                return decision['content']
            
            # Execute tool
            tool_name = decision['tool']
            tool_args = decision['args']
            
            self.trace.append({
                'step': 'thought',
                'content': decision.get('thought', '')
            })
            self.trace.append({
                'step': 'action',
                'tool': tool_name,
                'args': tool_args
            })
            
            result = self.tools[tool_name].execute(**tool_args)
            
            self.trace.append({
                'step': 'observation',
                'result': result
            })
            
            observations.append({'tool': tool_name, 'args': tool_args, 'result': result})
        
        return self._synthesize(query, observations)
    
    def show_trace(self):
        """Display the execution trace."""
        colors = {'query': '\033[94m', 'thought': '\033[93m',
                  'action': '\033[91m', 'observation': '\033[92m',
                  'answer': '\033[95m'}
        reset = '\033[0m'
        
        for entry in self.trace:
            step = entry['step']
            if step == 'query':
                print(f"Query: {entry['content']}")
            elif step == 'thought':
                print(f"  Thought: {entry['content']}")
            elif step == 'action':
                args_str = ', '.join(f'{k}="{v}"' for k, v in entry['args'].items())
                print(f"  Action: {entry['tool']}({args_str})")
            elif step == 'observation':
                result = entry['result']
                result_str = json.dumps(result, indent=None)[:120]
                print(f"  Observation: {result_str}")
            elif step == 'answer':
                print(f"  Thought: {entry.get('thought', '')}")
                print(f"  Answer: {entry['content'][:200]}")


# Test the ReAct agent
agent = ReActAgent(tools)

queries = [
    "Tell me about the transformer architecture",
    "What is relu?",
    "Tell me about RLHF and how it works",
]

for query in queries:
    print("\n" + "=" * 60)
    answer = agent.run(query)
    agent.show_trace()
    print("=" * 60)

### Visualization: ReAct Execution Flow

In [ ]:
# Visualize a ReAct trace
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
ax.set_xlim(0, 14)
ax.set_ylim(0, 7)
ax.axis('off')
ax.set_title('ReAct Execution Trace', fontsize=14, fontweight='bold')

steps = [
    (1, 5.5, 'Query', '#95a5a6', '"Tell me about\ntransformers"'),
    (3.5, 5.5, 'Thought', '#f39c12', '"I need to search\nfor this topic"'),
    (6, 5.5, 'Action', '#e74c3c', 'search("transformer\narchitecture")'),
    (8.5, 5.5, 'Observation', '#2ecc71', '"Introduced in 2017\nby Vaswani et al..."'),
    (11, 5.5, 'Thought', '#f39c12', '"I have enough\ninfo to answer"'),
]

steps2 = [
    (3.5, 2.5, 'Answer', '#9b59b6', '"The Transformer was\nintroduced in 2017..."'),
]

for x, y, label, color, text in steps + steps2:
    box = mpatches.FancyBboxPatch((x - 1, y - 0.6), 2, 1.2, boxstyle="round,pad=0.15",
                                   facecolor=color, edgecolor='black', linewidth=1.5, alpha=0.9)
    ax.add_patch(box)
    ax.text(x, y + 0.2, label, ha='center', va='center', fontsize=9,
            fontweight='bold', color='white')
    ax.text(x, y - 0.25, text, ha='center', va='center', fontsize=7, color='white')

# Arrows
for i in range(len(steps) - 1):
    ax.annotate('', xy=(steps[i+1][0] - 1, steps[i+1][1]),
               xytext=(steps[i][0] + 1, steps[i][1]),
               arrowprops=dict(arrowstyle='->', lw=2, color='gray'))

# Arrow to answer
ax.annotate('', xy=(steps2[0][0], steps2[0][1] + 0.6),
           xytext=(steps[-1][0], steps[-1][1] - 0.6),
           arrowprops=dict(arrowstyle='->', lw=2, color='gray',
                          connectionstyle='arc3,rad=0.3'))

# Labels
ax.text(7, 4.2, 'Reasoning + Acting Loop', ha='center', fontsize=11,
        style='italic', color='gray')

plt.tight_layout()
plt.show()

---

## 4. Chain-of-Thought Reasoning

**Chain-of-thought (CoT)** prompting dramatically improves reasoning by making the model "think step by step." For agents, CoT serves as the planning mechanism:

- **Without CoT**: LLM jumps to an action immediately (often wrong)
- **With CoT**: LLM reasons about what it knows, what it needs, then acts

### Why CoT Works for Agents

1. **Decomposition**: Breaks complex tasks into manageable steps
2. **Self-correction**: The model can catch its own errors mid-reasoning
3. **Tool selection**: Explicit reasoning helps choose the right tool
4. **Transparency**: Users can understand *why* the agent took an action

In [ ]:
# Demonstrate CoT benefit with a multi-step problem

class PlanningAgent:
    """Agent that creates an explicit plan before acting."""
    
    def __init__(self, tools):
        self.tools = tools
    
    def plan(self, query):
        """Create a step-by-step plan (simulated)."""
        query_lower = query.lower()
        plan_steps = []
        
        # Analyze what's needed
        needs_search = any(w in query_lower for w in ['tell me', 'how', 'what', 'explain', 'when'])
        needs_calc = any(c in query for c in '0123456789+-*/') or 'calculate' in query_lower
        needs_lookup = any(w in query_lower for w in ['define', 'definition'])
        
        if needs_search:
            # Extract key topics
            topics = [w for w in query_lower.split() if len(w) > 3 and w not in 
                     {'tell', 'about', 'what', 'how', 'does', 'this', 'that', 'with', 'from'}]
            plan_steps.append({
                'step': 1,
                'description': f'Search for information about: {", ".join(topics[:3])}',
                'tool': 'search',
                'args': {'query': ' '.join(topics[:3])}
            })
        
        if needs_lookup:
            terms = re.findall(r'define\s+(\w+)', query_lower)
            if terms:
                plan_steps.append({
                    'step': len(plan_steps) + 1,
                    'description': f'Look up definition of: {terms[0]}',
                    'tool': 'lookup',
                    'args': {'term': terms[0]}
                })
        
        if needs_calc:
            expr = re.findall(r'[\d\+\-\*/\(\)\. \*\*]+', query)
            if expr:
                plan_steps.append({
                    'step': len(plan_steps) + 1,
                    'description': f'Calculate: {expr[0].strip()}',
                    'tool': 'calculator',
                    'args': {'expression': expr[0].strip()}
                })
        
        plan_steps.append({
            'step': len(plan_steps) + 1,
            'description': 'Synthesize findings into a coherent answer',
            'tool': None,
            'args': {}
        })
        
        return plan_steps
    
    def execute_plan(self, query):
        """Plan then execute."""
        plan = self.plan(query)
        results = []
        
        print(f"Query: {query}")
        print(f"\nPlan ({len(plan)} steps):")
        for step in plan:
            print(f"  Step {step['step']}: {step['description']}")
        
        print(f"\nExecution:")
        for step in plan:
            if step['tool'] and step['tool'] in self.tools:
                result = self.tools[step['tool']].execute(**step['args'])
                results.append(result)
                print(f"  Step {step['step']}: {step['tool']}() -> {json.dumps(result)[:100]}")
            else:
                print(f"  Step {step['step']}: Synthesizing answer...")
        
        return results


planning_agent = PlanningAgent(tools)

print("=" * 60)
planning_agent.execute_plan("Tell me about attention in transformers")
print("\n" + "=" * 60)
planning_agent.execute_plan("What is backpropagation and how does it relate to gradient descent?")

---

## 5. Agent Architectures Compared

Different agent architectures suit different tasks:

| Architecture | Description | Best For |
|-------------|-------------|----------|
| **ReAct** | Interleave reasoning and acting | General tool use |
| **Plan-then-Execute** | Full plan upfront, then execute | Well-defined tasks |
| **Reflexion** | Act, evaluate, reflect, retry | Tasks needing self-correction |
| **Tree of Thoughts** | Explore multiple reasoning paths | Complex problem solving |
| **Multi-Agent** | Multiple specialized agents collaborate | Complex workflows |

In [ ]:
# Visualize agent architecture comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# ReAct: Linear interleave
ax = axes[0]
ax.set_xlim(0, 4)
ax.set_ylim(0, 8)
ax.axis('off')
ax.set_title('ReAct', fontsize=13, fontweight='bold')

react_steps = [
    (2, 7, 'Think', '#f39c12'),
    (2, 5.5, 'Act', '#e74c3c'),
    (2, 4, 'Observe', '#2ecc71'),
    (2, 2.5, 'Think', '#f39c12'),
    (2, 1, 'Answer', '#9b59b6'),
]
for x, y, label, color in react_steps:
    box = mpatches.FancyBboxPatch((x - 0.8, y - 0.35), 1.6, 0.7,
                                   boxstyle="round,pad=0.1", facecolor=color,
                                   edgecolor='black', linewidth=1.5)
    ax.add_patch(box)
    ax.text(x, y, label, ha='center', va='center', fontsize=10, fontweight='bold', color='white')

for i in range(len(react_steps) - 1):
    ax.annotate('', xy=(2, react_steps[i+1][1] + 0.35),
               xytext=(2, react_steps[i][1] - 0.35),
               arrowprops=dict(arrowstyle='->', lw=1.5, color='gray'))

# Plan-then-Execute
ax = axes[1]
ax.set_xlim(0, 4)
ax.set_ylim(0, 8)
ax.axis('off')
ax.set_title('Plan-then-Execute', fontsize=13, fontweight='bold')

box = mpatches.FancyBboxPatch((0.5, 6), 3, 1.2, boxstyle="round,pad=0.1",
                               facecolor='#f39c12', edgecolor='black', linewidth=1.5)
ax.add_patch(box)
ax.text(2, 6.6, 'Plan', ha='center', va='center', fontsize=11, fontweight='bold', color='white')

for i, y in enumerate([4.5, 3.2, 1.9]):
    box = mpatches.FancyBboxPatch((0.5, y), 3, 0.7, boxstyle="round,pad=0.1",
                                   facecolor='#e74c3c', edgecolor='black', linewidth=1.5)
    ax.add_patch(box)
    ax.text(2, y + 0.35, f'Execute Step {i+1}', ha='center', va='center',
            fontsize=9, fontweight='bold', color='white')

ax.annotate('', xy=(2, 5.2), xytext=(2, 6),
           arrowprops=dict(arrowstyle='->', lw=1.5, color='gray'))
for y1, y2 in [(4.5, 3.2), (3.2, 1.9)]:
    ax.annotate('', xy=(2, y2 + 0.7), xytext=(2, y1),
               arrowprops=dict(arrowstyle='->', lw=1.5, color='gray'))

box = mpatches.FancyBboxPatch((0.5, 0.5), 3, 0.7, boxstyle="round,pad=0.1",
                               facecolor='#9b59b6', edgecolor='black', linewidth=1.5)
ax.add_patch(box)
ax.text(2, 0.85, 'Answer', ha='center', va='center', fontsize=10, fontweight='bold', color='white')
ax.annotate('', xy=(2, 1.2), xytext=(2, 1.9),
           arrowprops=dict(arrowstyle='->', lw=1.5, color='gray'))

# Reflexion
ax = axes[2]
ax.set_xlim(0, 4)
ax.set_ylim(0, 8)
ax.axis('off')
ax.set_title('Reflexion', fontsize=13, fontweight='bold')

refl_steps = [
    (2, 7, 'Act', '#e74c3c'),
    (2, 5.5, 'Evaluate', '#3498db'),
    (2, 4, 'Reflect', '#f39c12'),
    (2, 2.5, 'Retry', '#e74c3c'),
    (2, 1, 'Answer', '#9b59b6'),
]

for x, y, label, color in refl_steps:
    box = mpatches.FancyBboxPatch((x - 0.8, y - 0.35), 1.6, 0.7,
                                   boxstyle="round,pad=0.1", facecolor=color,
                                   edgecolor='black', linewidth=1.5)
    ax.add_patch(box)
    ax.text(x, y, label, ha='center', va='center', fontsize=10, fontweight='bold', color='white')

for i in range(len(refl_steps) - 1):
    ax.annotate('', xy=(2, refl_steps[i+1][1] + 0.35),
               xytext=(2, refl_steps[i][1] - 0.35),
               arrowprops=dict(arrowstyle='->', lw=1.5, color='gray'))

# Loop arrow for reflexion
ax.annotate('', xy=(3.3, 7), xytext=(3.3, 4),
           arrowprops=dict(arrowstyle='->', lw=1.5, color='#f39c12',
                          connectionstyle='arc3,rad=-0.5'))
ax.text(3.8, 5.5, 'retry', fontsize=8, color='#f39c12', rotation=90, va='center')

plt.suptitle('Agent Architecture Patterns', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## 6. Agent Safety and Failure Modes

Agents are powerful but dangerous — they can take **irreversible actions**. Understanding failure modes is critical.

| Failure Mode | Example | Mitigation |
|-------------|---------|------------|
| **Infinite loops** | Agent keeps searching without converging | Max step limit |
| **Wrong tool** | Uses calculator when should search | Better tool descriptions |
| **Hallucinated actions** | Calls a tool that doesn't exist | Strict tool validation |
| **Unsafe actions** | Deletes files, sends emails | Permission system, sandboxing |
| **Goal drift** | Wanders from original task | Task decomposition, checkpoints |
| **Cascading errors** | Error in step 1 propagates | Error handling, retries |

In [ ]:
class SafeAgent:
    """Agent with safety guardrails."""
    
    def __init__(self, tools, max_steps=5, max_tool_calls=10):
        self.tools = tools
        self.max_steps = max_steps
        self.max_tool_calls = max_tool_calls
        self.tool_call_count = 0
        self.errors = []
    
    def validate_tool_call(self, tool_name, args):
        """Validate a tool call before execution."""
        # Check tool exists
        if tool_name not in self.tools:
            return False, f"Unknown tool: {tool_name}"
        
        # Check rate limit
        if self.tool_call_count >= self.max_tool_calls:
            return False, "Tool call limit exceeded"
        
        # Check required parameters
        tool = self.tools[tool_name]
        for param in tool.parameters:
            if param not in args:
                return False, f"Missing required parameter: {param}"
        
        return True, "OK"
    
    def safe_execute(self, tool_name, **args):
        """Execute a tool with error handling."""
        valid, message = self.validate_tool_call(tool_name, args)
        if not valid:
            self.errors.append(message)
            return {'status': 'blocked', 'reason': message}
        
        try:
            self.tool_call_count += 1
            result = self.tools[tool_name].execute(**args)
            return result
        except Exception as e:
            self.errors.append(str(e))
            return {'status': 'error', 'error': str(e)}


# Demonstrate safety features
safe = SafeAgent(tools, max_tool_calls=3)

print("Safety demonstrations:")
print("\n1. Valid tool call:")
print(f"   {safe.safe_execute('calculator', expression='2+2')}")

print("\n2. Invalid tool name:")
print(f"   {safe.safe_execute('delete_database', target='all')}")

print("\n3. Missing parameters:")
print(f"   {safe.safe_execute('calculator')}")

# Exhaust rate limit
safe.safe_execute('calculator', expression='1+1')
safe.safe_execute('calculator', expression='1+1')
print("\n4. Rate limit exceeded:")
print(f"   {safe.safe_execute('calculator', expression='1+1')}")

print(f"\nErrors logged: {safe.errors}")

---

## 7. Agent Evaluation Metrics

How do we measure if an agent is good? Key metrics:

| Metric | What It Measures | How |
|--------|-----------------|-----|
| **Task completion** | Did the agent finish the task? | Binary success/fail |
| **Accuracy** | Was the answer correct? | Compare to ground truth |
| **Efficiency** | How many steps/tool calls? | Count steps |
| **Tool selection** | Did it use the right tools? | Compare to optimal trace |
| **Error recovery** | Did it handle failures gracefully? | Inject errors, check recovery |

In [ ]:
class AgentEvaluator:
    """Evaluate agent performance on a benchmark."""
    
    def __init__(self):
        self.results = []
    
    def evaluate(self, agent, test_cases):
        """Run agent on test cases and measure performance."""
        self.results = []
        
        for test in test_cases:
            agent.trace = []
            answer = agent.run(test['query'])
            
            # Measure metrics
            trace = agent.trace
            n_steps = len([t for t in trace if t['step'] == 'action'])
            tools_used = [t['tool'] for t in trace if t['step'] == 'action']
            
            # Check if expected tool was used
            correct_tool = test.get('expected_tool') in tools_used if test.get('expected_tool') else True
            
            # Check if answer contains expected content
            answer_correct = any(
                keyword.lower() in answer.lower()
                for keyword in test.get('expected_keywords', [])
            )
            
            self.results.append({
                'query': test['query'],
                'n_steps': n_steps,
                'tools_used': tools_used,
                'correct_tool': correct_tool,
                'answer_correct': answer_correct,
                'completed': len(answer) > 10,
            })
        
        return self._compute_metrics()
    
    def _compute_metrics(self):
        n = len(self.results)
        return {
            'completion_rate': sum(r['completed'] for r in self.results) / n,
            'accuracy': sum(r['answer_correct'] for r in self.results) / n,
            'tool_accuracy': sum(r['correct_tool'] for r in self.results) / n,
            'avg_steps': np.mean([r['n_steps'] for r in self.results]),
        }


# Evaluation benchmark
test_cases = [
    {'query': 'Tell me about transformers',
     'expected_tool': 'search', 'expected_keywords': ['attention', '2017']},
    {'query': 'What is relu?',
     'expected_tool': 'lookup', 'expected_keywords': ['max', 'activation']},
    {'query': 'How does RLHF work?',
     'expected_tool': 'search', 'expected_keywords': ['reward', 'PPO']},
    {'query': 'What is backpropagation?',
     'expected_tool': 'search', 'expected_keywords': ['gradient', 'chain']},
    {'query': 'Tell me about embeddings and semantic similarity',
     'expected_tool': 'search', 'expected_keywords': ['vector', 'cosine']},
]

evaluator = AgentEvaluator()
agent = ReActAgent(tools)
metrics = evaluator.evaluate(agent, test_cases)

print("Agent Evaluation Results:")
for metric, value in metrics.items():
    print(f"  {metric}: {value:.2%}" if 'rate' in metric or 'accuracy' in metric
          else f"  {metric}: {value:.1f}")

# Visualize
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
metric_names = list(metrics.keys())
metric_values = list(metrics.values())
colors = ['#2ecc71', '#3498db', '#f39c12', '#e74c3c']
bars = ax.bar(metric_names, metric_values, color=colors, edgecolor='black', alpha=0.8)

for bar, val in zip(bars, metric_values):
    label = f'{val:.0%}' if val <= 1 else f'{val:.1f}'
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            label, ha='center', fontsize=11, fontweight='bold')

ax.set_ylabel('Score', fontsize=12)
ax.set_title('Agent Performance Metrics', fontsize=13, fontweight='bold')
ax.set_xticklabels(metric_names, rotation=15, ha='right')
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

---

## 8. Multi-Agent Systems

Complex tasks often benefit from **multiple specialized agents** working together:

- **Researcher**: Gathers information from search/documents
- **Coder**: Writes and executes code
- **Reviewer**: Checks work quality
- **Orchestrator**: Coordinates the team

In [ ]:
class SpecializedAgent:
    """A specialized agent with a specific role."""
    
    def __init__(self, name, role, tools):
        self.name = name
        self.role = role
        self.tools = tools
    
    def process(self, task, context=None):
        """Process a task given optional context from other agents."""
        results = []
        for tool_name, args in self._decide_actions(task, context):
            if tool_name in self.tools:
                result = self.tools[tool_name].execute(**args)
                results.append({'tool': tool_name, 'result': result})
        return {'agent': self.name, 'role': self.role, 'results': results}
    
    def _decide_actions(self, task, context):
        """Decide which tools to use (simplified)."""
        actions = []
        if self.role == 'researcher':
            actions.append(('search', {'query': task}))
        elif self.role == 'fact_checker':
            # Check key terms from context
            if context:
                for r in context.get('results', []):
                    if 'result' in r and 'results' in r.get('result', {}):
                        for item in r['result']['results']:
                            key = item.get('title', '').lower()
                            actions.append(('lookup', {'term': key}))
                            break
        return actions


class Orchestrator:
    """Coordinates multiple specialized agents."""
    
    def __init__(self, agents):
        self.agents = {a.name: a for a in agents}
    
    def run_pipeline(self, task):
        """Run agents in a pipeline."""
        print(f"Orchestrator: Task = '{task}'\n")
        
        # Step 1: Research
        researcher = self.agents.get('researcher')
        if researcher:
            research_output = researcher.process(task)
            print(f"  {researcher.name} ({researcher.role}):")
            for r in research_output['results']:
                print(f"    Tool: {r['tool']} -> {json.dumps(r['result'])[:100]}")
        
        # Step 2: Fact check
        checker = self.agents.get('fact_checker')
        if checker:
            check_output = checker.process(task, context=research_output)
            print(f"  {checker.name} ({checker.role}):")
            for r in check_output['results']:
                print(f"    Tool: {r['tool']} -> {json.dumps(r['result'])[:100]}")
        
        print(f"\n  Orchestrator: Pipeline complete.")
        return {'research': research_output, 'fact_check': check_output if checker else None}


# Build multi-agent system
researcher = SpecializedAgent('researcher', 'researcher', tools)
checker = SpecializedAgent('fact_checker', 'fact_checker', tools)

orchestrator = Orchestrator([researcher, checker])
result = orchestrator.run_pipeline("Explain how attention works in transformers")

---

## Exercises

### Exercise 1: Weather Agent

Create a `WeatherTool` (simulated) and integrate it into the ReAct agent. The tool should accept a city name and return simulated weather data. Test with queries like "What's the weather in San Francisco?" and "Should I bring an umbrella in New York?"

In [ ]:
# Exercise 1: Your code here
# Hint: Create a WeatherTool class that extends Tool,
# add it to the tools dict, and modify the agent's reasoning


### Exercise 2: Reflexion Agent

Implement a Reflexion agent that: (1) attempts to answer, (2) self-evaluates its answer, (3) reflects on what went wrong, (4) retries with improved approach. Test on a math word problem.

In [ ]:
# Exercise 2: Your code here
# Hint: After the agent generates an answer, add a "self-check" step
# that verifies the answer and triggers a retry if needed


### Exercise 3: Agent Benchmark Suite

Create a benchmark of 10+ test cases with varying difficulty. Compare ReAct vs PlanningAgent on completion rate, accuracy, and efficiency. Which architecture performs better on which types of tasks?

In [ ]:
# Exercise 3: Your code here


---

## Summary

### Key Concepts

- **AI agents** use LLMs as reasoning engines that can take actions through tools
- **Tools** are the agent's interface to the world: search, calculate, execute code, call APIs
- **ReAct** interleaves reasoning and acting for flexible problem-solving
- **Chain-of-thought** reasoning improves tool selection and task decomposition
- **Safety guardrails** (rate limits, validation, sandboxing) are essential for production agents
- **Multi-agent systems** let specialized agents collaborate on complex tasks
- Agent evaluation requires task-specific metrics: completion, accuracy, efficiency

### Fundamental Insight

Agents transform LLMs from passive text generators into active problem-solvers. The key is not the tools themselves — it's the reasoning loop that decides *which* tool to use and *when*. The better the reasoning, the more capable the agent. This is why improvements in base LLM reasoning (from GPT-3 to GPT-4 to Claude) translate directly into more capable agents.

---

## Next Steps

Building agents is one thing — knowing if they actually work is another. In **Notebook 23: Evaluating AI Systems**, we'll learn the science of AI evaluation: benchmarks, automated eval frameworks, LLM-as-judge, red teaming, and the metrics that determine whether an AI system is ready for production.